# Monte Carlo Option Pricing: Convergence & Black–Scholes Validation

This notebook analyzes Monte Carlo convergence for European call option pricing under a risk-neutral GBM model.
We study how simulation count affects accuracy and variance, and benchmark results against the Black–Scholes
closed-form solution.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from src.gbm import simulate_gbm_terminal
from src.option_pricing import discounted_payoff_call, estimate_mean_se_ci
from src.black_scholes import black_scholes_call


## Single Monte Carlo Run

We first compute a single Monte Carlo estimate using antithetic variates to reduce variance and compare it to
the analytical Black–Scholes price.


In [ ]:
S0 = 100
K = 100
r = 0.03
sigma = 0.2
T = 1.0
steps = 252
N = 20_000

ST = simulate_gbm_terminal(S0, r, sigma, T, steps, N, seed=42, antithetic=True)
disc = discounted_payoff_call(ST, K, r, T)
price, se, ci = estimate_mean_se_ci(disc)

bs = black_scholes_call(S0, K, r, sigma, T)

price, se, ci, bs


## Convergence as Simulation Count Increases

We compute Monte Carlo estimates for increasing numbers of simulations and examine how:
- price estimates stabilize,
- standard error decreases approximately at a 1/sqrt(N) rate,
- results converge to Black–Scholes.


In [ ]:
simulation_counts = [500, 1000, 2500, 5000, 10_000, 25_000, 50_000]

mc_prices = []
mc_ses = []
av_prices = []
av_ses = []

for N in simulation_counts:
    ST_mc = simulate_gbm_terminal(S0, r, sigma, T, steps, N, seed=42, antithetic=False)
    disc_mc = discounted_payoff_call(ST_mc, K, r, T)
    price_mc, se_mc, _ = estimate_mean_se_ci(disc_mc)

    ST_av = simulate_gbm_terminal(S0, r, sigma, T, steps, N, seed=42, antithetic=True)
    disc_av = discounted_payoff_call(ST_av, K, r, T)
    price_av, se_av, _ = estimate_mean_se_ci(disc_av)

    mc_prices.append(price_mc)
    mc_ses.append(se_mc)
    av_prices.append(price_av)
    av_ses.append(se_av)


In [ ]:
plt.figure()
plt.plot(simulation_counts, mc_prices, marker="o", label="MC")
plt.plot(simulation_counts, av_prices, marker="o", label="Antithetic MC")
plt.axhline(bs, linestyle="--", label="Black–Scholes")
plt.xscale("log")
plt.xlabel("Number of Simulations")
plt.ylabel("Call Price")
plt.title("Monte Carlo Convergence")
plt.legend()
plt.show()

plt.figure()
plt.plot(simulation_counts, mc_ses, marker="o", label="MC SE")
plt.plot(simulation_counts, av_ses, marker="o", label="Antithetic SE")
plt.xscale("log")
plt.yscale("log")
plt.xlabel("Number of Simulations")
plt.ylabel("Standard Error")
plt.title("Standard Error Decay")
plt.legend()
plt.show()


## Interpretation

As the number of simulations increases, Monte Carlo estimates converge toward the Black–Scholes benchmark.
Standard error decays at approximately a 1/sqrt(N) rate, confirming the Central Limit Theorem.

Antithetic variates consistently reduce variance, producing smaller standard errors for the same N.

These experiments validate both the numerical implementation and the theoretical pricing model.


## Sensitivity experiments (financial intuition)

We vary volatility and maturity to confirm expected economic behavior:
- Higher volatility → higher call price
- Longer maturity → higher call price


In [ ]:
def price_once(S0, K, r, sigma, T, steps=252, N=50_000):
    ST = simulate_gbm_terminal(S0, r, sigma, T, steps, N, seed=42, antithetic=True)
    disc = discounted_payoff_call(ST, K, r, T)
    p, se, ci = estimate_mean_se_ci(disc)
    bs = black_scholes_call(S0, K, r, sigma, T)
    return p, se, ci, bs

for sig in [0.15, 0.20, 0.30]:
    p, se, ci, bs = price_once(S0, K, r, sig, T)
    print(f"sigma={sig:.2f} | MC={p:.4f} (SE={se:.4f}) | BS={bs:.4f} | AbsErr={abs(p-bs):.4f}")


## Historical Volatility Calibration from Market Data

To connect the model to real data, we estimate annualized historical volatility from daily SPY returns.
This calibrated volatility is then used as an input to the Monte Carlo pricer and Black–Scholes formula.

This demonstrates how market data can inform model parameters rather than assuming values.


In [ ]:
import yfinance as yf

df = yf.download("SPY", start="2015-01-01", progress=False)

df.to_csv("data/spy_prices.csv")

df.head()

In [ ]:
prices = df["Adj Close"] if "Adj Close" in df.columns else df["Close"]

log_returns = np.log(prices / prices.shift(1)).dropna()

log_returns.describe()


In [ ]:
daily_vol = log_returns.std()
annual_vol = daily_vol * np.sqrt(252)

daily_vol, annual_vol


In [ ]:
sigma_hist = annual_vol

S0 = prices.iloc[-1]
K = S0
r = 0.03
T = 1.0
steps = 252
N = 50_000

ST = simulate_gbm_terminal(S0, r, sigma_hist, T, steps, N, seed=42, antithetic=True)
disc = discounted_payoff_call(ST, K, r, T)
price_mc, se, ci = estimate_mean_se_ci(disc)

bs_hist = black_scholes_call(S0, K, r, sigma_hist, T)

price_mc, se, ci, bs_hist


## Interpretation (Historical Calibration)

Using volatility estimated from SPY returns produces option prices consistent between Monte Carlo
and Black–Scholes models.

This experiment shows how real market data can be incorporated into simulation-based pricing frameworks.


## Conclusions

This project implemented a Monte Carlo pricer for European options under a risk-neutral GBM model,
validated convergence to the Black–Scholes analytical solution, and demonstrated the effectiveness
of antithetic variates for variance reduction.

Sensitivity experiments confirmed economically intuitive behavior: option values increase with
volatility and maturity.

